# Using CDP Databases
Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

### Connecting to the database

CDP Seattle uses Firebase's 'Cloud Firestore' to store our data. However, a properly setup database host and associated database module _should_ have the same functionality.

Here is how to connect to the Seattle database for **read only** operations.

**Note:** This notebook connects to the staging instance of Seattle's Firestore database. To use production data, connect to `cdp-seattle`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase

db = CloudFirestoreDatabase("stg-cdp-seattle")
db

<CloudFirestoreDatabase [stg-cdp-seattle]>

### Retrieving a single item
If you know the id of an item in a table, please use the `select_row_by_id` function provided.

In [2]:
event = db.select_row_by_id(table="event", id="5970315e-cc33-4d84-8ee0-131274f2076d")
event

{'event_id': '5970315e-cc33-4d84-8ee0-131274f2076d',
 'body_id': 'e3f298eb-bb75-4273-af45-16c3f0e6f962',
 'legistar_event_link': 'https://seattle.legistar.com/MeetingDetail.aspx?LEGID=3964&GID=393&G=FFE3B678-CEF6-4197-84AC-5204EA4CFC0C',
 'source_uri': 'http://www.seattlechannel.org/FullCouncil?videoid=x104672',
 'legistar_event_id': 3964,
 'event_datetime': datetime.datetime(2019, 5, 28, 14, 0),
 'agenda_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/5/3964_A_City_Council_19-05-28_Full_Council_Meeting_Agenda.pdf',
 'minutes_file_uri': None,
 'video_uri': 'http://video.seattle.gov:8080/media/council/council_052819_2021941V.mp4',
 'created': datetime.datetime(2019, 6, 4, 0, 47, 47, 28966)}

### Retrieving many items from a table

You may not know the id's of items you are looking for. In that case, use the `select_rows_as_list` function provided.

In [3]:
events = db.select_rows_as_list(table="event")
events[0]

{'event_id': '0855398b-ea28-4053-a866-f40c09d11d94',
 'legistar_event_id': 3972,
 'event_datetime': datetime.datetime(2019, 6, 3, 9, 30),
 'agenda_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/6/3972_A_Council_Briefing_19-06-03_Council_Briefing.pdf',
 'minutes_file_uri': None,
 'video_uri': 'http://video.seattle.gov:8080/media/council/brief_060319_2011943V.mp4',
 'created': datetime.datetime(2019, 6, 4, 1, 0, 32, 901831),
 'legistar_event_link': 'https://seattle.legistar.com/MeetingDetail.aspx?LEGID=3972&GID=393&G=FFE3B678-CEF6-4197-84AC-5204EA4CFC0C',
 'body_id': '87c4988e-522d-4cd6-81e3-b066c3747236',
 'source_uri': 'http://www.seattlechannel.org/CouncilBriefings?videoid=x104707'}

### Joining with other tables

In the above event results, notice that a `body_id` is returned for each event. To attach body details to this we can use the python package `pandas` and query the `body` table. Let's first put each of the query results into `pandas.DataFrame` objects.

In [4]:
import pandas as pd

In [5]:
events = pd.DataFrame(events)
events.head()

,agenda_file_uri,body_id,created,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri
0,http://legistar2.granicus.com/seattle/meetings...,87c4988e-522d-4cd6-81e3-b066c3747236,2019-06-04 01:00:32.901831,2019-06-03 09:30:00,0855398b-ea28-4053-a866-f40c09d11d94,3972,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/CouncilBriefings...,http://video.seattle.gov:8080/media/council/br...
1,http://legistar2.granicus.com/seattle/meetings...,87c4988e-522d-4cd6-81e3-b066c3747236,2019-06-04 00:40:32.910534,2019-05-28 09:30:00,47fb22e5-93f8-4e2f-a707-1152fe671a0c,3966,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/CouncilBriefings...,http://video.seattle.gov:8080/media/council/br...
2,http://legistar2.granicus.com/seattle/meetings...,e3f298eb-bb75-4273-af45-16c3f0e6f962,2019-06-04 00:47:47.028966,2019-05-28 14:00:00,5970315e-cc33-4d84-8ee0-131274f2076d,3964,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
3,http://legistar2.granicus.com/seattle/meetings...,dd8af69a-52ae-4d0e-8cce-33494a71d89f,2019-06-04 00:40:53.106354,2019-05-28 10:30:00,7060eac6-0f96-44da-86c9-6f408335f451,3965,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/selectcommitteem...,http://video.seattle.gov:8080/media/council/ho...
4,http://legistar2.granicus.com/seattle/meetings...,d83f0c8e-6b4d-474e-985a-44f15a539da3,2019-06-04 00:47:48.074184,2019-05-24 12:30:00,8505f961-c918-4782-b684-8aba372e09a4,3955,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/su...


In [6]:
bodies = db.select_rows_as_list("body")
bodies = pd.DataFrame(bodies)
bodies.head()

,body_id,created,description,name
0,87c4988e-522d-4cd6-81e3-b066c3747236,2019-06-04 00:40:31.798002,None,Council Briefing
1,ced0a843-a095-4df5-a916-72de9eba775d,2019-06-04 00:36:13.194924,None,"Civil Rights, Utilities, Economic Development,..."
2,d83f0c8e-6b4d-474e-985a-44f15a539da3,2019-06-04 00:47:47.700167,None,Sustainability and Transportation Committee
3,dd8af69a-52ae-4d0e-8cce-33494a71d89f,2019-06-04 00:40:51.977643,None,Select Committee on Citywide Mandatory Housing...
4,e3f298eb-bb75-4273-af45-16c3f0e6f962,2019-06-04 00:03:50.029975,None,City Council


In [7]:
expanded_event_details = events.merge(bodies, left_on="body_id", right_on="body_id", suffixes=("_event", "_body"))
expanded_event_details.head()

,agenda_file_uri,body_id,created_event,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description,name
0,http://legistar2.granicus.com/seattle/meetings...,87c4988e-522d-4cd6-81e3-b066c3747236,2019-06-04 01:00:32.901831,2019-06-03 09:30:00,0855398b-ea28-4053-a866-f40c09d11d94,3972,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/CouncilBriefings...,http://video.seattle.gov:8080/media/council/br...,2019-06-04 00:40:31.798002,None,Council Briefing
1,http://legistar2.granicus.com/seattle/meetings...,87c4988e-522d-4cd6-81e3-b066c3747236,2019-06-04 00:40:32.910534,2019-05-28 09:30:00,47fb22e5-93f8-4e2f-a707-1152fe671a0c,3966,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/CouncilBriefings...,http://video.seattle.gov:8080/media/council/br...,2019-06-04 00:40:31.798002,None,Council Briefing
2,http://legistar2.granicus.com/seattle/meetings...,e3f298eb-bb75-4273-af45-16c3f0e6f962,2019-06-04 00:47:47.028966,2019-05-28 14:00:00,5970315e-cc33-4d84-8ee0-131274f2076d,3964,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...,2019-06-04 00:03:50.029975,None,City Council
3,http://legistar2.granicus.com/seattle/meetings...,e3f298eb-bb75-4273-af45-16c3f0e6f962,2019-06-04 00:03:50.288189,2019-06-03 14:00:00,f39c25f4-1d9e-40f4-97b1-910189bc1fe8,3975,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...,2019-06-04 00:03:50.029975,None,City Council
4,http://legistar2.granicus.com/seattle/meetings...,dd8af69a-52ae-4d0e-8cce-33494a71d89f,2019-06-04 00:40:53.106354,2019-05-28 10:30:00,7060eac6-0f96-44da-86c9-6f408335f451,3965,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/selectcommitteem...,http://video.seattle.gov:8080/media/council/ho...,2019-06-04 00:40:51.977643,None,Select Committee on Citywide Mandatory Housing...


`left_on` refers to the column name in the dataframe calling the operation.
In this case, the column to merge on is `body_id` in the events results.

Similarly, `right_on` refers to the column name in the dataframe to be passed to the operation.
In this case, the column to merge on is `id` in the bodies results.

`suffixes` is a tuple to use for adding suffixes to any columns with the same name between the two dataframes.
Commonly for CDP query results, these are columns such as `created`, which provide a `datetime` value for when that row was stored in the database.

Please refer to `pandas.DataFrame.merge` documentation for more details.

[reference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge)

### Filtering

You may notice that the function: `select_rows_as_list` allows for additional parameters to be passed: `filters`, `order_by`, and `limit`. Unfortunately, at this time, `filters` is not available for the open access portions of the API. So while you can provide them to the function, they are not actually used. Because of this, you must do filtering on your end. Fortunately however, `pandas` works well for these types of operations.

[stackoverflow](https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas)

In [8]:
mha = "Select Committee on Citywide Mandatory Housing Affordability (MHA)"
mha_events = expanded_event_details.loc[expanded_event_details["name"] == mha]
mha_events

,agenda_file_uri,body_id,created_event,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description,name
4,http://legistar2.granicus.com/seattle/meetings...,dd8af69a-52ae-4d0e-8cce-33494a71d89f,2019-06-04 00:40:53.106354,2019-05-28 10:30:00,7060eac6-0f96-44da-86c9-6f408335f451,3965,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/selectcommitteem...,http://video.seattle.gov:8080/media/council/ho...,2019-06-04 00:40:51.977643,None,Select Committee on Citywide Mandatory Housing...
